# Webscraping 

En este tutorial vamos a analizar el webscraping, una técnica muy utilizada en ciencia de datos para recolectar datos de páginas webs.


## ¿Qué es el Webscraping?

[Webscraping](https://es.wikipedia.org/wiki/Web_scraping) consiste en utilizar una herramienta informática (como puede ser un lenguaje de programación) para extraer datos de una página web de forma automática. Básicamente utiliza peticiones HTTP para "pedir" una página web de forma similar a como haríamos con un navegador web (como Firefox, Chrome o Internet Explorer). Una vez hecha dicha petición, extrae la información que nos interesa y la guarda (en un archivo o en una base de datos).


<center><img src='https://qph.cf2.quoracdn.net/main-qimg-819ffb7ec3f7f53563f97d057b32a391-pjlq' width="400"></center>

## Scrapeando datos de Google Play

<center><img src='https://i.blogs.es/3bce76/valoracion/450_1000.jpg' width="400"></center>


Para entender cómo hacer webscraping, vamos a desarrollar un código en Python para extraer información sobre apps disponibles en **Google Play**, la plataforma de distribución digital de aplicaciones móviles para los dispositivos con sistema operativo Android. Esta plataforma permite a los usuarios navegar y descargar aplicaciones, juegos, música, libros, revistas y películas.

En particular, vamos a tratar obtener información de la app de Yuka. https://play.google.com/store/apps/details?id=io.yuka.android



<center><img src='https://play-lh.googleusercontent.com/fmGc9NGmf0ZTV2PQIhZ7lprvqNaUOaZEFXZg6MIG5fDaUiTua4Y2D7IxA3jaHDj2xQ=w240-h480-rw' width="200"></center>


## Acceder a una web con `Requests`

Lo primero que necesitamos es una biblioteca capaz de hacer estas peticiones HTTP (Hypertext Transfer Protocol), es decir, que se conecte a la página web que queremos, y nos "traiga" a Python el contenido de dicha web.

Para esta operación, vamos a utilizar una biblioteca muy sencilla y potente: [`Requests`](http://docs.python-requests.org/en/master/). 


In [26]:
# !pip install requests

import requests

Para pedir una web, solo tenemos que decirle a `Requests` que haga lo que se llama una petición **HTTP GET**. 


In [27]:
url = 'https://play.google.com/store/apps/details?id=io.yuka.android'
response = requests.get(url)

In [28]:
response

<Response [200]>

Vamos a ver el tipo de nuestra variable `google_play`:

In [29]:
type(response)

requests.models.Response

In [30]:
# Muestra el código HTTP respuesta o Status de google_play:

response.status_code

200

El método nos devuelve el número 200, lo que significa que la petición es correcta. En caso de introducir una URL que no existe, obtendríamos un código 404. Existen otros códigos de respuesta, que puedes encontrar [aquí](https://es.wikipedia.org/wiki/Anexo:C%C3%B3digos_de_estado_HTTP).


## Contenido de una respuesta HTTP

Una página web no deja de ser código. A pesar de que nosotros vemos la web de forma bonita y llena de imágenes y texto legible, las páginas están escritas en principalmente tres lenguajes de programación: [HTML](https://es.wikipedia.org/wiki/HTML), [CSS](https://es.wikipedia.org/wiki/Hoja_de_estilos_en_cascada) y [Javascript](https://es.wikipedia.org/wiki/JavaScript).

Cuando pedimos una página web con nuestro navegador, éste recibe el código de la página. Una vez recibido, el navegador "dibuja" o *renderiza* ese código para mostrarnos las páginas web bonitas.

`Requests` no hace este útlimo paso: únicamente recibe el código de la página (llamado *código fuente*), con el que vamos a interactuar para hacer nuestro scraping.

Como prueba, vamos a hacer una cosa: vamos a abrir con nuestro navegador web favorito la [página](https://play.google.com/store/apps/details?id=com.spotify.music) de la app de Spotify y vamos a hacer click derecho, por ejemplo, en el nombre de la app y hacemos click en *Inspeccionar elemento*. Debería desplegarse una ventanita donde aparece el código fuente (en HTML principalmente) de esa parte de la página web.

`Requests` recibe el código fuente en lo que se llama *el contenido de la respuesta* tras la petición HTTP GET. Para mostrar este contenido, utlizamos el método `.text.`Nos va a mostrar un código larguísimo y nada apetecible...

In [ ]:
response.text

Ahí está todo: todo el texto, todas las imágenes... Nuestra tarea es seleccionar de todo ese código las partes que nos interesan para nuestro proyecto

## Analizar código HTML con `BeautifulSoup`
`BeautifulSoup` es una biblioteca muy famosa para hacer web scraping. Nos permite analizar código HTML y extraer datos de ellos de forma sencilla.

In [ ]:
from bs4 import BeautifulSoup 

Para que `BeautifulSoup` sea capaz de el código de la web es necesario crear una instancia de la clase `BeautifulSoup`. Dicha instancia debe ser creada con dos argumentos posicionales:

1. El primero deber ser el código fuente de la web en string (¡lo tenemos!)
2. El segundo ha de ser un string que le diga a `BeautifulSoup` el *parser* (o procesador) a utilizar. La biblioteca puede utilizar [unos cuantos](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser). Nosotros utilizaremos el `"html.parser"`.


In [32]:
yuka_bs = BeautifulSoup(response.text, parser='html.parser')

In [ ]:
# Utilizar el método prettify() para mostrar el código de forma más estética
yuka_bs.prettify()

## Brevísima introducción a HTML

`yuka_bs` es una instancia de `BeautifulSoup`, lista para "darnos lo que le pidamos" de todo el código fuente de la página web.

Ahora solo queda decirle qué queremos...

Esta parte es más complicada si no sabes HTML.

HTML es un lenguaje de marcas. Los elementos que podemos ver en una página web (como son títulos, párrafos, imágenes...) tienen nombres definidos. Por ejemplo:

+ Los títulos grandes son `h1`
+ Un párrafo es `p`
+ Una imagen es `img`
+ Una URL es `a`

La tónica es casi siempre la misma: en HTML debemos "meter" el contenido de un elemento entre *marcas*. Por ejemplo, para hacer un título sería:

```html
<h1>Esto es un título.</h1>
```

Los elementos de HTML pueden tener dos cosas llamadas `id` y `class`. Esto permite a los propios desarrolladores poner algo de orden dentro de lo que es el caos de las páginas web modernas. Por ejemplo: una página como la de Google Play utliza cientos de `<span>` y párrafos...

Pues bien, cada elemento puede tener un `id` único si queremos que sea diferenciable del resto de elementos del mismo tipo. Gracias a esto, podemos diferenciar (tanto nosotros como los ordenadores):

```html
<p id="comentario_1">Primer comentario</p>
```

De:

```html
<p id="comentario_2">Otro comentario</p>
```

Asimismo, elementos "similares" o del mismo estilo/aspecto suelen tener una misma `class` (no confundir con las clases de Python):

```html
<h1 class="titulo_grande">Un título</h1>
```
Y: 

```html
<h1 class="titulo_grande">Otro título</h1>
```

## Extraer los datos que nos interesan

A modo de ejemplo, vamos a extraer los siguientes datos de interés:

- Nombre de la app
- Rating
- Logo 
- Apps relacionadas

Cualquier instancia de `BeautifulSoup` tiene el método [`.find()`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find) que permite buscar cualquier tipo de elemento HTML, y se queda con la primera ocurrencia. Este método toma como argumentos interesantes:

1. Un posicional, que es un string con el nombre del elemento a buscar (como pueden ser `"h1"`, `"p"` o `"div"`).
2. Un argumento opcional llamado `id`, que casualmente permite especificar más en la búsqueda; buscando solo elementos con el `id` especificado como string.
3. Otro argumento opcional llamado `class_` (así con barra baja para diferenciarlo de la keyword `class` que sirve para definir clases en Python). Al igual que `id`, permite limitar la búsqueda a los elementos del HTML que tengan dicha `class`.

Por ejemplo: si queremos buscar el primer elemento `h2` que tenga `id="123"` y `class="front-title-spacing"`, haríamos:

```python
mi_instancia_de_BeatutifulSoup.find("h2", id="123", class_="front-title-spacing")
```

### Nombre de app

In [34]:
titulo = yuka_bs.find('h1', class_='Fd93Bb F5UCq p5VxAd')

In [35]:
titulo

<h1 class="Fd93Bb F5UCq p5VxAd" itemprop="name"><span>Yuka - Food &amp; cosmetic scan</span></h1>

In [36]:
titulo = titulo.get_text()
titulo

'Yuka - Food & cosmetic scan'

### Rating

In [37]:
rating = yuka_bs.find('div', class_='TT9eCd')

In [38]:
rating.get_text()

'4.7star'

In [39]:
rating = float(rating.get_text().replace('star', ''))
rating

4.7

### Logo

In [40]:
logo = yuka_bs.find('div', class_='Il7kR')

In [42]:
logo.find('img')


<img alt="Icon image" aria-hidden="true" class="T75of cN0oRe fFmL2e" itemprop="image" src="https://play-lh.googleusercontent.com/fmGc9NGmf0ZTV2PQIhZ7lprvqNaUOaZEFXZg6MIG5fDaUiTua4Y2D7IxA3jaHDj2xQ=w240-h480" srcset="https://play-lh.googleusercontent.com/fmGc9NGmf0ZTV2PQIhZ7lprvqNaUOaZEFXZg6MIG5fDaUiTua4Y2D7IxA3jaHDj2xQ=w480-h960 2x"/>

In [43]:
logo = logo.find('img')['src']
logo

'https://play-lh.googleusercontent.com/fmGc9NGmf0ZTV2PQIhZ7lprvqNaUOaZEFXZg6MIG5fDaUiTua4Y2D7IxA3jaHDj2xQ=w240-h480'

### Otras apps

In [44]:
apps = yuka_bs.find_all('a', class_='Si6A0c nT2RTe')

In [46]:
apps

[<a class="Si6A0c nT2RTe" href="/store/apps/details?id=com.app.tgtg" jslog="38003; 1:500|CAIaUgoUEhIKDGNvbS5hcHAudGd0ZxABGAMQADITCMyXsIzAyoEDFYGWCgkdI/AJ9XITCI+A8vekyoEDFciVCgkdBlYLrooBDQgAEgkKBWVuLVVTEACqAlcaVQgAEhQKEgoMY29tLmFwcC50Z3RnEAEYA0oTCMyXsIzAyoEDFYGWCgkdI/AJ9ZoBEwiPgPL3pMqBAxXIlQoJHQZWC676AQ8KDQgAEgkKBWVuLVVTEAA=; track:click,impression"><div class="j2FCNc cQv9D"><img alt="Thumbnail image" aria-hidden="true" class="T75of stzEZd" loading="lazy" src="https://play-lh.googleusercontent.com/Ht7OXfuYPvtSdFl7PnuX6KisTLgSceF9krOmxof7klTgqNR7UBVc53z63Vm3NuOQ0eo=s64" srcset="https://play-lh.googleusercontent.com/Ht7OXfuYPvtSdFl7PnuX6KisTLgSceF9krOmxof7klTgqNR7UBVc53z63Vm3NuOQ0eo=s128 2x"/><div class="cXFu1"><div class="ubGTjb"><span class="DdYX5">Too Good To Go: End Food Waste</span></div><div class="ubGTjb"><span class="wMUdtb">Too Good To Go Aps</span></div><div class="ubGTjb"><div aria-label="Rated 4.8 stars out of five stars" style="display: inline-flex; align-items: center;"><spa

In [45]:
len(apps)

6

In [47]:
# Para acceder a la url, busco el elemento 'href'
apps[0]['href']

'/store/apps/details?id=com.app.tgtg'

In [ ]:
# Tenemos que añadirle el inicio de la url
cons= 'https://play.google.com'

In [48]:
app_list = []
for app in apps:
    url = cons + app['href']
    app_list.append(url)

In [49]:
app_list

['https://play.google.com/store/apps/details?id=com.app.tgtg',
 'https://play.google.com/store/apps/details?id=com.lafourchette.lafourchette',
 'https://play.google.com/store/apps/details?id=com.hellofresh.androidapp',
 'https://play.google.com/store/apps/details?id=com.wishop.dev.jow',
 'https://play.google.com/store/apps/details?id=com.sillens.shapeupclub',
 'https://play.google.com/store/apps/details?id=com.myfitnesspal.android']

## Unir todo en un dataframe

In [50]:
import pandas as pd
row = [titulo, rating, logo, app_list]
app_df = pd.DataFrame([row], columns = ['name','rating','logo','related_apps'])

In [51]:
app_df

,name,rating,logo,related_apps
0,Yuka - Food & cosmetic scan,4.7,https://play-lh.googleusercontent.com/fmGc9NGm...,[https://play.google.com/store/apps/details?id...
